In [1]:
import openai
import json
import os 

from dotenv import load_dotenv
from polywrap_client import PolywrapClient
from typing import cast

from pathlib import Path
from polywrap_client import PolywrapClient, ClientConfig
from polywrap_core import Uri, InvokerOptions, UriPackageOrWrapper, UriResolver
from polywrap_client_config_builder import PolywrapClientConfigBuilder
from polywrap_uri_resolvers import FsUriResolver,SimpleFileReader, StaticResolver, RecursiveResolver
from polywrap_uri_resolvers import UriResolverAggregator
from polywrap_http_plugin import http_plugin


import asyncio

import nest_asyncio

nest_asyncio.apply()

load_dotenv()

ipfs_wrapper_path = Path("fs//Users/robertohenriquez/pycode/polywrap/hackathon/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/ipfs-http-client")


resolver = RecursiveResolver(
        UriResolverAggregator(
            [
                cast(UriResolver, FsUriResolver(file_reader=SimpleFileReader())),
                cast(UriResolver, StaticResolver({Uri("wrap://ens/wraps.eth:http@1.1.0", ipfs_wrapper_path): http_plugin()})),
            ]
        )
    )
config = ClientConfig(resolver=resolver)
client = PolywrapClient(config)

openai.api_key = os.getenv("OPENAI_API_KEY")





In [2]:
import json
import openai

def polywrap_bignumber(base_number, factor):
    """Multiplies a base number by a factor to get a new number"""
    uri = Uri.from_str(
        f'fs//Users/robertohenriquez/pycode/cloud/AGI/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/bignumber'
    )
    args = {
        "arg1": str(base_number),  # The base number
        "obj": {
            "prop1": str(factor),  # multiply the base number by this factor
        },
    }
    options: InvokerOptions[UriPackageOrWrapper] = InvokerOptions(
        uri=uri, method="method", args=args, encode_result=False
    )
    print(asyncio.run(client.invoke(options)))
    result = asyncio.run(client.invoke(options))
    return f"the result is {result}"

def fetch_tool_library(tool_name):
    # Mapping of keywords to tools
    tool_mappings = {
        "ipfs": ["wrap/ipfs"],
        "http": ["wrap/http"],
        "ens": ["wrap/ens"],
        "ethers": ["wrap/ethers"],
        "ethereum": ["wrap/ethers"],
    }
    
    # Check if the tool_name has any keywords
    keywords = tool_name.lower().split()
    matching_tools = []
    
    for keyword in keywords:
        if keyword in tool_mappings:
            matching_tools.extend(tool_mappings[keyword])
    
    # Remove duplicates and sort the tools alphabetically
    matching_tools = sorted(list(set(matching_tools)))
    
    return str(matching_tools)


def invoke_tool(tool_uri, arguments):
    # Replace this with your implementation of invoking the tool
    return f"Invoking tool with URI: {tool_uri}, Arguments: {arguments}"



In [3]:
def run_conversation(question, functions):
    messages = [
        {"role": "user", "content": question}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    if message.get("function_call"):
        print("-> Using a function call now")
        function_name = message["function_call"]["name"]
        function_args_string = message["function_call"]["arguments"]
        function_args = json.loads(function_args_string)

        function_response = globals()[function_name](**function_args)

        messages.append({"role": "assistant", "content": function_response})

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )

        return second_response
    else:
        raise Exception("The wrapper was not invoked")


In [4]:
question = "Find what tools are available for ipfs"
# Define the functions for our ChatGPT Agent

functions = [
    {
        "name": "polywrap_bignumber",
        "description": "Use this function every time you need to make a multiplication",
        "parameters": {
            "type": "object",
            "properties": {
                "base_number": {
                    "type": "number",
                    "description": "The base number",
                },
                "factor": {
                    "type": "number",
                    "description": "The factor to multiply the base number",
                },
            },
            "required": ["base_number", "factor"],
        },
    },
    {
        "name": "fetch_tool_library",
        "description": "Fetches information about a specific tool from the tool library, given the tool name.",
        "parameters": {
            "type": "object",
            "properties": {
                "tool_name": {
                    "type": "string",
                    "description": "The name of the tool to fetch",
                },
            },
            "required": ["tool_name"],
        },
    },
    {
        "name": "invoke_tool",
        "description": "Invokes the given tool with the specified arguments",
        "parameters": {
            "type": "object",
            "properties": {
                "tool_uri": {
                    "type": "string",
                    "description": "The URI of the tool to invoke",
                },
                "arguments": {
                    "type": "object",
                    "description": "The arguments to pass to the tool",
                },
            },
            "required": ["tool_uri", "arguments"],
        },
    },
]

print(run_conversation(question, functions))


-> Using a function call now
{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"tool_name\": \"ipfs\"\n}",
    "name": "fetch_tool_library"
  },
  "role": "assistant"
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "There are several tools available for IPFS (InterPlanetary File System):\n\n1. IPFS CLI (Command-Line Interface) - This is the official command-line tool for interacting with IPFS. It allows you to manage IPFS nodes, add and retrieve files, resolve content addresses, and perform various other operations.\n\n2. IPFS Companion - This is a browser extension that enhances your web browsing experience by adding IPFS integration. It allows you to access IPFS-hosted websites directly from your browser, and also provides other useful features like automatic URL rewriting, IPNS support, and more.\n\n3. WebUI - IPFS WebUI is a user-friendly web-based interface for IPFS. It provides a graphical interface f

In [8]:
question = "Find what tools are available for http in our skill library"
print(run_conversation(question, functions))


-> Using a function call now
{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"tool_name\": \"http\"\n}",
    "name": "fetch_tool_library"
  },
  "role": "assistant"
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 'wrap/http' tool is available in our skill library for HTTP.",
        "role": "assistant"
      }
    }
  ],
  "created": 1687350408,
  "id": "chatcmpl-7TrD64q81G5ceywUWW86zQT2VGgff",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 26,
    "total_tokens": 41
  }
}


In [6]:
question = "Find what tools are available for ENS"
print(run_conversation(question, functions))

-> Using a function call now
{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"tool_name\": \"ENS\"\n}",
    "name": "fetch_tool_library"
  },
  "role": "assistant"
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 'wrap/ens' tool is available for working with ENS (Ethereum Name Service).",
        "role": "assistant"
      }
    }
  ],
  "created": 1687350292,
  "id": "chatcmpl-7TrBE1ilZlbok1rwWDMhk4zhbPsFE",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 24,
    "total_tokens": 45
  }
}


In [7]:
question = "Find what tools are available for ethereum in the skill library"
print(run_conversation(question, functions))

-> Using a function call now
{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"tool_name\": \"ethereum\"\n}",
    "name": "fetch_tool_library"
  },
  "role": "assistant"
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Some tools available for Ethereum in the skill library are:\n\n1. Web3.js: A JavaScript library that allows interaction with Ethereum nodes through HTTP or IPC connections.\n2. Truffle: A development framework for Ethereum that helps in building, deploying, and testing smart contracts.\n3. Ganache: A personal blockchain for Ethereum development that allows developers to test and deploy smart contracts on a local network.\n4. Remix: An online IDE (Integrated Development Environment) for Ethereum smart contract development.\n5. Hardhat: A development environment for Ethereum that allows for fast and reliable smart contract testing and deployment.\n6. ethers.js: A TypeScript library for Ethereu